In [1]:
pip install Pillow torch torchvision torchaudio requests geopy SpeechRecognition pytesseract pandas numpy easyocr

Note: you may need to restart the kernel to use updated packages.


In [2]:
from PIL import Image
import os
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import requests
import json
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import speech_recognition as sr
from datetime import datetime
import easyocr  # Import the easyocr library

# Initialize easyocr reader ONCE
try:
    reader = easyocr.Reader(['en'])  # You can add other languages here, e.g., ['en', 'hi']
except Exception as e:
    print(f"Error initializing easyocr: {e}")
    reader = None

# Initialize geolocator
geolocator = Nominatim(user_agent="multimedia_report_generator")

# Initialize speech recognizer
r = sr.Recognizer()

# Load pre-trained model and labels ONCE
model = None
transform = None
imagenet_classes = []
try:
    model = models.resnet18(pretrained=True)
    model.eval()
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    try:
        with open("imagenet_classes.txt", "r") as f:
            imagenet_classes = [s.strip() for s in f.readlines()]
    except FileNotFoundError:
        print("Warning: imagenet_classes.txt not found. Downloading...")
        url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
        try:
            response = requests.get(url)
            response.raise_for_status()
            imagenet_labels_json = response.json()
            if isinstance(imagenet_labels_json, dict):
                imagenet_classes = list(imagenet_labels_json.values())
            elif isinstance(imagenet_labels_json, list):
                imagenet_classes = imagenet_labels_json
            else:
                print("Error: Unexpected format for downloaded ImageNet labels.")
                imagenet_classes = ["unknown"] * 1000
            with open("imagenet_classes.txt", "w") as f:
                for label in imagenet_classes:
                    f.write(label + "\n")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading ImageNet labels: {e}")
            imagenet_classes = ["unknown"] * 1000
        except json.JSONDecodeError as e:
            print(f"Error decoding downloaded JSON: {e}")
            imagenet_classes = ["unknown"] * 1000
    except Exception as e:
        print(f"Error loading ImageNet class labels: {e}")
        imagenet_classes = ["unknown"] * 1000
except Exception as e:
    print(f"Error initializing PyTorch models or transforms: {e}")
    model = None
    transform = None
    imagenet_classes = ["Error"] * 1000

def get_input_paths():
    input_paths = {}

    # Event Name (Mandatory)
    event_name = input("Enter the name of the event: ")
    input_paths['event_name'] = event_name

    # Image Input (Mandatory)
    image_paths = []
    print("\n--- Image Input ---")
    while True:
        image_path = input("Enter the path to an image file (or type 'done' if you've entered all images): ")
        if image_path.lower() == 'done':
            break
        if os.path.exists(image_path):
            image_paths.append(image_path)
        else:
            print(f"Error: Image path '{image_path}' does not exist.")
    if not image_paths:
        print("Error: At least one image is required.")
        return None
    input_paths['images'] = image_paths

    # College Name (Mandatory)
    college_name_input = input("Enter the name of the college: ")
    input_paths['college_name'] = college_name_input

    # Optional Inputs
    print("\n--- Optional Inputs ---")
    analyze_objects = input("Do you want to analyze objects in the image? (yes/no): ").lower()
    input_paths['analyze_objects'] = analyze_objects == 'yes'

    recognize_text = input("Do you want to recognize text in the image? (yes/no): ").lower()
    input_paths['recognize_text'] = recognize_text == 'yes'

    add_audio = input("Do you want to provide an audio file or speak now? (file/live/no): ").lower()
    if add_audio == 'file':
        audio_path = input("Enter the path to the audio file: ")
        if os.path.exists(audio_path):
            input_paths['audio'] = audio_path
        else:
            print(f"Error: Audio path '{audio_path}' does not exist.")
    elif add_audio == 'live':
        input_paths['audio'] = 'live'  # Use a special string to indicate live input

    add_feedback = input("Do you want to provide feedback text? (yes/no): ").lower()
    if add_feedback == 'yes':
        feedback_text = input("Enter the feedback text: ")
        input_paths['feedback'] = feedback_text

    add_location = input("Do you want to provide location information? (yes/no): ").lower()
    if add_location == 'yes':
        location_input = input("Enter the location (e.g., latitude,longitude or a place name): ")
        input_paths['location'] = location_input

    return input_paths

def process_image_objects(image_path):
    if model is None or transform is None or not imagenet_classes or "Error" in imagenet_classes:
        return "Error: PyTorch model or dependencies not initialized correctly."
    try:
        img = Image.open(image_path).convert('RGB')
        img_t = transform(img)
        batch_t = torch.unsqueeze(img_t, 0)

        with torch.no_grad():
            output = model(batch_t)

        _, indices = torch.sort(output, descending=True)
        probabilities = torch.nn.functional.softmax(output, dim=1)[0]

        top5_preds = [(imagenet_classes[idx], probabilities[idx].item() * 100) for idx in indices[0][:5]]
        description = f"The image likely contains: {', '.join([f'{pred[0]} ({pred[1]:.2f}%)' for pred in top5_preds])}."
        return description
    except FileNotFoundError:
        return f"Error: Image file not found at '{image_path}'."
    except Exception as e:
        return f"Error processing image '{image_path}' for object detection: {e}"

def process_image_text(image_path):
    if reader is None:
        return "Error: easyocr not initialized."
    try:
        result = reader.readtext(image_path)
        if result:
            recognized_text = "\n".join([detection[1] for detection in result])
            return f"Recognized text:\n{recognized_text}"
        else:
            return "No text recognized in the image."
    except FileNotFoundError:
        return f"Error: Image file not found at '{image_path}' for text recognition."
    except Exception as e:
        return f"Error processing image '{image_path}' for text recognition with easyocr: {e}"

# Consider implementing a simple cache for geocoding if repeated locations are expected
# geocode_cache = {}

def get_college_name_from_location(location_str):
    # if location_str in geocode_cache:
    #     return geocode_cache[location_str]
    try:
        college = None
        if "," in location_str:
            lat_str, lon_str = location_str.split(',')
            try:
                latitude = float(lat_str.strip())
                longitude = float(lon_str.strip())
                location = geolocator.reverse((latitude, longitude), exactly_one=True, language="en")
                if location and location.raw.get('address'):
                    address = location.raw['address']
                    college = address.get('university') or address.get('college')
            except ValueError:
                return "Error: Invalid latitude/longitude format."
        else:
            location = geolocator.geocode(location_str, exactly_one=True, language="en")
            if location and location.raw.get('address'):
                address = location.raw['address']
                college = address.get('university') or address.get('college')

        result = college if college else "[College name not found based on location]"
        # geocode_cache[location_str] = result
        return result
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        # geocode_cache[location_str] = f"Error with geocoding service: {e}"
        return f"Error with geocoding service: {e}"
    except Exception as e:
        # geocode_cache[location_str] = f"An unexpected error occurred during geocoding: {e}"
        return f"An unexpected error occurred during geocoding: {e}"

def process_audio(audio_input):
    if audio_input == 'live':
        with sr.Microphone() as source:
            print("Speak now...")
            try:
                audio_data = r.listen(source)
                text = r.recognize_google(audio_data)
                return f"Live audio transcript: {text}"
            except sr.UnknownValueError:
                return "Could not understand audio."
            except sr.RequestError as e:
                return f"Could not request results from Google Speech Recognition service; {e}"
            except Exception as e:
                return f"Error processing live audio: {e}"
    else:
        try:
            with sr.AudioFile(audio_input) as source:
                audio_data = r.record(source)
                try:
                    text = r.recognize_google(audio_data)
                    return f"Audio transcript: {text}"
                except sr.UnknownValueError:
                    return "Could not understand audio from file."
                except sr.RequestError as e:
                    return f"Could not request results from Google Speech Recognition service (file); {e}"
        except FileNotFoundError:
            return f"Error: Audio file not found at '{audio_input}'."
        except Exception as e:
            return f"Error processing audio file '{audio_input}': {e}"

def generate_report(inputs):
    report_content = ""

    event_name = inputs.get('event_name', '[Event Name not provided]')
    college_name = inputs.get('college_name', '[College Name not provided]')

    final_report = f"--- Report ---\n\n"
    final_report += f"Event: {event_name}\n"
    final_report += f"College: {college_name}\n\n"

    if 'images' in inputs and inputs['images']:
        final_report += "--- Image Analysis ---\n"
        for i, image_path in enumerate(inputs['images']):
            image_report = f"Image {i+1}:\n"
            if inputs.get('analyze_objects', False):
                object_description = process_image_objects(image_path)
                image_report += f"- Object Detection: {object_description}\n"
            if inputs.get('recognize_text', False):
                text_description = process_image_text(image_path)
                image_report += f"- Text Recognition: {text_description}\n"
            final_report += image_report + "\n"
        final_report += "\n"

    if 'location' in inputs:
        final_report += "--- Location Information ---\n"
        final_report += f"Provided Location: {inputs['location']}\n"
        # Only call geocoding if a college name wasn't directly provided or if you specifically need the location's college
        if college_name == '[College Name not provided]':
            college_name_from_location = get_college_name_from_location(inputs['location'])
            final_report += f"College Name (from location): {college_name_from_location}\n\n"
        else:
            final_report += f"College Name (from location): [Geocoding skipped as college name was provided]\n\n"

    if 'feedback' in inputs:
        final_report += "--- Feedback ---\n"
        final_report += f"User Feedback: {inputs['feedback']}\n\n"

    if 'audio' in inputs:
        final_report += "--- Audio Information ---\n"
        if inputs['audio'] == 'live':
            final_report += "Audio Source: Live Microphone Input\n"
        else:
            final_report += f"Audio Path: {inputs['audio']}\n"
        audio_transcript = process_audio(inputs['audio'])
        final_report += f"Audio Transcript: {audio_transcript}\n\n"

    final_report += "--- End of Report ---"

    return final_report

def save_report_to_file(report):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"report_{timestamp}.txt"
    try:
        with open(filename, "w") as f:
            f.write(report)
        print(f"\nReport saved to '{filename}'")
    except Exception as e:
        print(f"Error saving report to file: {e}")

if __name__ == "__main__":
    inputs = get_input_paths()
    if inputs and 'images' in inputs:
        report = generate_report(inputs)
        print("\n--- Generated Report ---")
        print(report)
        save_report_to_file(report)
    else:
        print("No valid inputs received. Cannot generate a report.")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

c:\Users\jyoth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jyoth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



--- Image Input ---

--- Optional Inputs ---

--- Generated Report ---
--- Report ---

Event: Algoveda
College: Vignana Bharathi Institute of Technology,Ghatkesar,Hyderabad

--- Image Analysis ---
Image 1:
- Object Detection: The image likely contains: stage (54.01%), balance beam (5.87%), scoreboard (5.40%), grand piano (3.82%), front curtain (3.38%).
- Text Recognition: Recognized text:
AlgoVeda
VBIT
AlgoVeda

Image 2:
- Object Detection: The image likely contains: stage (28.83%), balance beam (14.08%), scoreboard (14.04%), baseball player (3.85%), horizontal bar (3.24%).
- Text Recognition: Recognized text:
VBIT
AlgoVeda


--- Location Information ---
Provided Location: Vignana Bharathi Institution of Technology,Aushapur,Ghatkesar,Hyderabad,Telangana,India
College Name (from location): [Geocoding skipped as college name was provided]

--- Feedback ---
User Feedback: It was a nice coding competition

--- End of Report ---

Report saved to 'report_20250426_154812.txt'


In [6]:
pip install python-dotenv google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [11]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# Configure the Gemini API if the key is available
if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
    model_gemini = genai.GenerativeModel('gemini-pro')
else:
    print("Warning: GEMINI_API_KEY not found in .env file. Gemini functionality will be disabled.")
    model_gemini = None

def generate_report_with_gemini(report_data):
    """
    Generates a report using the Gemini API based on the provided data.

    Args:
        report_data (str): A string containing the structured information
                           to be included in the report.

    Returns:
        str: The generated report text, or None if an error occurred
             or the Gemini API key is not configured.
    """
    if model_gemini:
        prompt = f"Generate a concise report based on the following data:\n\n{report_data}"
        try:
            response = model_gemini.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"Error generating report with Gemini: {e}")
            return None
    else:
        return "Gemini API key not configured. Cannot generate report."

if __name__ == "__main__":
    # Example of report data you might have collected from other parts of your script
    report_information = f"""
    Event: Sample Event
    College: Example College
    Image Analysis: Image 1 - Likely contains: cat (90%), dog (5%).
                    Image 1 - Recognized text: Welcome!
    Location: Hyderabad, India
    Feedback: User liked the event.
    Audio Transcript: The speaker discussed the main points...
    """

    if model_gemini:
        generated_report = generate_report_with_gemini(report_information)
        if generated_report:
            print("\n--- Generated Report (using Gemini) ---")
            print(generated_report)
        else:
            print("Failed to generate report using Gemini.")
    else:
        print("Gemini API is not configured. Skipping report generation.")

Error generating report with Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]
Failed to generate report using Gemini.


In [ ]:
from datetime import datetime
import os
import random
import mysql.connector

# Database configuration
DB_HOST = "localhost"
DB_USER = "root"  # Replace with your MySQL username
DB_PASSWORD = "Adithya@2005"  # Replace with your MySQL password
DB_NAME = "project1"

def create_tables():
    try:
        mydb = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        mycursor = mydb.cursor()

        # Create input table if it doesn't exist
        mycursor.execute("""
            CREATE TABLE IF NOT EXISTS input (
                id INT AUTO_INCREMENT PRIMARY KEY,
                extracted_features_path VARCHAR(255),
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        """)

        # Create output table if it doesn't exist
        mycursor.execute("""
            CREATE TABLE IF NOT EXISTS output (
                id INT AUTO_INCREMENT PRIMARY KEY,
                input_id INT,
                report_content TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (input_id) REFERENCES input(id)
            )
        """)

        mydb.commit()
        mycursor.close()
        mydb.close()
        print("Database tables 'input' and 'output' created (if they didn't exist).")
    except mysql.connector.Error as err:
        print(f"Error creating tables: {err}")

def get_input_paths():
    input_paths = {}
    extracted_features_path = input("Enter the path to the file containing extracted features (.txt): ").strip()

    if not os.path.exists(extracted_features_path):
        print(f"Error: File not found at '{extracted_features_path}'.")
        return None
    input_paths['extracted_features_path'] = extracted_features_path
    return input_paths

def insert_input_data(inputs):
    try:
        mydb = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        mycursor = mydb.cursor()

        sql = "INSERT INTO input (extracted_features_path) VALUES (%s)"
        val = (inputs.get('extracted_features_path'),)
        mycursor.execute(sql, val)
        mydb.commit()
        input_id = mycursor.lastrowid
        mycursor.close()
        mydb.close()
        return input_id
    except mysql.connector.Error as err:
        print(f"Error inserting input data: {err}")
        return None

def insert_output_data(input_id, report):
    try:
        mydb = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        mycursor = mydb.cursor()

        sql = "INSERT INTO output (input_id, report_content) VALUES (%s, %s)"
        val = (input_id, report)
        mycursor.execute(sql, val)
        mydb.commit()
        mycursor.close()
        mydb.close()
        print("Report content saved to the 'output' table.")
    except mysql.connector.Error as err:
        print(f"Error inserting output data: {err}")

def generate_paragraph_report(inputs):
    extracted_features_path = inputs.get('extracted_features_path')
    report_paragraph = ""

    try:
        with open(extracted_features_path, "r") as f:
            extracted_features = [line.strip() for line in f.readlines() if line.strip()]

        if extracted_features:
            report_paragraph += "The analysis of the provided data reveals several key characteristics. "
            report_paragraph += ", ".join(extracted_features) + ". "

            # Generate AI content based on extracted features
            ai_elaboration = []
            for feature in extracted_features:
                if "image contains" in feature.lower():
                    entity = feature.split("contains")[-1].strip()
                    ai_elaboration.append(f"The presence of a '{entity}' suggests a visual focus on this element, potentially indicating its importance within the context.")
                elif "dominant color" in feature.lower():
                    color = feature.split("is")[-1].strip()
                    ai_elaboration.append(f"The dominant color being '{color}' could influence the overall mood or convey specific symbolic meanings.")
                elif "detected object" in feature.lower():
                    obj = feature.split(":")[-1].strip()
                    ai_elaboration.append(f"The detection of a '{obj}' as a significant object warrants further consideration of its role.")
                elif "confidence score" in feature.lower():
                    score = float(feature.split(':')[-1].strip())
                    if score > 0.8:
                        ai_elaboration.append(f"With a high confidence score of {score:.2f}, this particular observation is deemed highly reliable.")
                    else:
                        ai_elaboration.append(f"The confidence score of {score:.2f} suggests a moderate level of certainty for this finding.")
                else:
                    ai_elaboration.append(f"This feature contributes to the overall understanding and may hold further implications.")

            if ai_elaboration:
                report_paragraph += "Considering these individual findings, a broader interpretation emerges. "
                report_paragraph += " ".join(random.sample(ai_elaboration, min(len(ai_elaboration), 2))) + ". "

            report_paragraph = report_paragraph.strip()
            # Ensure at least 3 lines (by adding more sentences)
            while report_paragraph.count('.') < 2 and ai_elaboration:
                report_paragraph += random.choice(ai_elaboration) + ". "
                report_paragraph = report_paragraph.strip()

            if report_paragraph.count('.') < 2:
                report_paragraph += "In conclusion, the identified features collectively offer valuable insights into the subject matter."

        else:
            report_paragraph = "No extracted features were found in the provided file."

        return report_paragraph.strip()

    except FileNotFoundError:
        return "Error: Extracted features file not found."
    except Exception as e:
        return f"Error reading extracted features file: {e}"

def save_report_to_file(report):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"report_{timestamp}.txt"
    try:
        with open(filename, "w") as f:
            f.write(report)
        print(f"\nReport saved to '{filename}'")
    except Exception as e:
        print(f"Error saving report to file: {e}")

if __name__ == "__main__":
    # Create the tables if they don't exist
    create_tables()

    inputs = get_input_paths()
    if inputs:
        input_id = insert_input_data(inputs)
        if input_id:
            report = generate_paragraph_report(inputs)
            print("\n--- Generated Report ---")
            # Print the report with line breaks for better readability
            formatted_report = ""
            sentence_endings = ['.', '!', '?']
            current_line = ""
            for word in report.split():
                if len(current_line) + len(word) + 1 <= 70:  # Adjust line width as needed
                    current_line += (word + " ")
                else:
                    formatted_report += current_line.strip() + "\n"
                    current_line = word + " "
            formatted_report += current_line.strip()
            print(formatted_report)
            save_report_to_file(report)
            insert_output_data(input_id, report)
        else:
            print("Failed to record input data.")
    else:
        print("No valid input received.")